**The TensorFlow notebooks for this project are still in development so may contain bugs and are lacking in documentation. Please see the PyStan notebooks for a clearer walkthrough.**

In [41]:
import time

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp

In [42]:
tfd = tfp.distributions

In [43]:
# Confirm GPU in use
tf.config.list_physical_devices('GPU')

[]

In [66]:
# Parameters
P = 10
S = 10 ** 2 # Total population
NA = 30 # Number of surveys
SEED = 1729

In [67]:
# Ground truth
np.random.seed(SEED)
true_transmission_rate = np.random.beta(2, 10, P)
true_occurrence_rate = np.random.beta(2, 10, P)
base_rate = np.random.beta(2, 10, 1)

t_i = np.random.beta(8, 2, 1)  # Prob(tested | infected)
t_not_i = np.random.beta(2, 20, 1)  # Prob(tested | not-infected)
true_gamma = np.hstack([t_i, t_not_i])

In [69]:
# Simulate data
data = {}
for p in range(P):
    occurrence = np.random.binomial(1, true_occurrence_rate[p], S)
    transmission = occurrence * np.random.binomial(1, true_transmission_rate[p], S)
    data[f'O{p+1}'] = occurrence
    data[f'T{p+1}'] = transmission

data['T0'] = np.random.binomial(1, base_rate, S)
X = pd.DataFrame(data)
z = X.loc[:, X.columns.str.startswith('T')].sum(axis=1)
y = (z > 0).astype(int)

# Resampling using testing probabilites conditional on infected
tested = y*np.random.binomial(1, true_gamma[0], S) + (1-y)*np.random.binomial(1, true_gamma[1], S)
y = y[tested == 1]
if NA >= (S - y.shape[0]):
    X_survey = X[tested == 0].reset_index()
else:
    X_survey = X[tested == 0].reset_index().sample()
X = X[tested == 1].reset_index()

N = X.shape[0]
NA = X_survey.shape[0]

X = X.loc[:, X.columns.str.startswith('O')]
X_survey = X_survey.loc[:, X_survey.columns.str.startswith('O')]

In [70]:
# Convert to tensors
X = tf.convert_to_tensor(X, dtype=tf.float32)
X_survey = tf.convert_to_tensor(X_survey, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)
# Move to GPU
X = X + tf.fill(X.shape, 0.0)
X_survey = X_survey + tf.fill(X_survey.shape, 0.0)
y = y + tf.fill(y.shape, 0.0)

In [77]:
# Define log-likelihood
@tf.function
def censored_poisbinom_loglike(theta, rho, gamma):
    target = 0
    # Pre-computation
    log1m_theta = tf.math.log(1-theta)
    log_gamma = tf.math.log(gamma)
    log1m_gamma = tf.math.log(1-gamma)
    # Support
    if tf.math.reduce_any(tf.math.logical_or(theta <= 0., theta >= 1.)):
        return -np.inf
    if tf.math.logical_or(rho <= 0., rho >= 1.):
        return -np.inf
    if tf.math.reduce_any(tf.math.logical_or(gamma <= 0., gamma >= 1.)):
        return -np.inf
    # Likelihood (Survey)
    s = tf.einsum('ij,j->i', X_survey, log1m_theta) + tf.math.log(1-rho)
    target += tf.math.reduce_sum(tfp.math.log_add_exp(
            tfp.math.log1mexp(s) + log1m_gamma[0],
            s + log1m_gamma[1]
    ))
    # Likelihood (Tests)
    s = tf.einsum('ij,j->i', X, log1m_theta) + tf.math.log(1-rho)
    target += tf.math.reduce_sum(tf.where(
        y == 1,
        tfp.math.log1mexp(s) + log_gamma[0], 
        s + log_gamma[1]
    ))
    return target

In [78]:
# Define negative log-likelihood and use AD to compute gradients
@tf.function
def censored_poisbinom_negloglike(params):
    theta, rho, gamma = tf.split(params, [P, 1, 2], axis=0)
    # need to take these back down to vectors and scalars:
    theta = tf.reshape(theta,(P,))
    rho = tf.reshape(rho,())
    gamma = tf.reshape(gamma, (2,))
    return -1 * censored_poisbinom_loglike(theta, rho, gamma)

@tf.function
def censored_poisbinom_negloglike_and_grad(params):
    return tfp.math.value_and_gradient(
        censored_poisbinom_negloglike, 
        params
    )

In [79]:
# Approximate MLE using gradient descent
start = tf.fill(P + 3, 0.5)

optim_results = tfp.optimizer.bfgs_minimize(
    censored_poisbinom_negloglike_and_grad, start, tolerance=1e-8
)

est_params = optim_results.position.numpy()
est_serr = np.sqrt(np.diagonal(optim_results.inverse_hessian_estimate.numpy()))

In [80]:
# Set model parameters
nuts_samples = 1000
nuts_burnin = 200
init_step_size=.3
init = [est_params[:P], est_params[P], est_params[P+1:]]

In [81]:
# Fit model
@tf.function
def nuts_sampler(init):
    nuts_kernel = tfp.mcmc.NoUTurnSampler(
        target_log_prob_fn=censored_poisbinom_loglike, 
        step_size=init_step_size,
    )
    adapt_nuts_kernel = tfp.mcmc.DualAveragingStepSizeAdaptation(
        inner_kernel=nuts_kernel,
        num_adaptation_steps=nuts_burnin,
        step_size_getter_fn=lambda pkr: pkr.step_size,
        log_accept_prob_getter_fn=lambda pkr: pkr.log_accept_ratio,
        step_size_setter_fn=lambda pkr, new_step_size: pkr._replace(step_size=new_step_size)
    )

    samples = tfp.mcmc.sample_chain(
        num_results=nuts_samples,
        current_state=init,
        kernel=adapt_nuts_kernel,
        num_burnin_steps=nuts_burnin,
        parallel_iterations=10,
        trace_fn=None
    )
    return samples

start = time.time()
samples = nuts_sampler(init)
print(f"{time.time() - start:.02f} seconds elapsed")

6.84 seconds elapsed
